In [3]:
import csv
import spacy
import numpy as np
import random
from sklearn.svm import SVC
from collections import OrderedDict

### Generate datasets
1. Sentences that are itineraries for training -- Here we generate only sentences that are identifical and has three different structures
> Je veux aller de depart à destination
<br>
> Trajet de depart à destination
<br>
> Aller-retour de depart à destination

In [59]:
train_sentences = []
test_sentences = []
train_labels = []
test_labels = []

with open('stops.txt', encoding='UTF-8') as file_stops:
            data_stops = csv.reader(file_stops, delimiter=',')
            next(data_stops)
            list_stops=[]
            for row in data_stops:
                # stop_name = row[1].strip().replace(' ', '_')
                # list_stops.append(stop_name)
                list_stops.append(row[1].strip())

        # Get all the cities of France 
with open('commune_2022.csv', encoding='ISO-8859-1') as file_cities : 
    data_cities = csv.reader(file_cities, delimiter=',')
    next(data_cities)
    
    list_cities=[]
    for row in data_cities:
        # city_name = row[6].strip().replace(' ', '_')
        # list_cities.append(city_name)
        list_cities.append(row[6].strip())

i = 0
for x in range(0, 100):
    if x<= 30:
        if i == 0:
            sentence = 'Trajet de {} à {}'.format(list_cities[random.randint(0, len(list_cities)-1)], list_cities[random.randint(0, len(list_cities)-1)+1])
            i = i+1
        elif i == 1:
            sentence = 'Je veux aller de {} à {}'.format(list_cities[random.randint(0, len(list_cities)-1)], list_cities[random.randint(0, len(list_cities)-1)+1])
            i = i+1
        elif i == 2:
            sentence = 'Aller-retour de {} à {}'.format(list_cities[random.randint(0, len(list_cities)-1)], list_cities[random.randint(0, len(list_cities)-1)+1])
            i = i+1
        elif i == 3:
            sentence = 'Je veux aller de {} à {}'.format(list_stops[random.randint(0,len(list_stops)-1)], list_stops[random.randint(0, len(list_stops)-1)+1])
            i = i+1
        elif i == 4:
            sentence = 'Trajet de {} à {}'.format(list_stops[random.randint(0,len(list_stops)-1)], list_stops[random.randint(0, len(list_stops)-1)+1])
            i = i+1
        else:
            sentence = 'Aller-retour de {} à {}'.format(list_stops[random.randint(0,len(list_stops)-1)], list_stops[random.randint(0, len(list_stops)-1)+1])
            i = 0

        test_sentences.append(sentence.lower())
        test_labels.append(1)

    else: 
        if i == 0:
            sentence = 'Je veux aller de depart à destination'
            i = i+1
        elif i == 1:
            sentence = 'Trajet de depart à destination'
            i = i+1
        else:
            sentence = 'Aller-retour de depart à destination'
            i = 0

        train_sentences.append(sentence.lower())
        train_labels.append(1)
    
## Add to the same array, the sentences that are not itineraries
with open('random_not_itinerary.csv', encoding="UTF-8") as file_non_itinerary:
    data_not_itinerary = csv.reader(file_non_itinerary, delimiter=',')
    i = 0
    for row in data_not_itinerary:
        i=i+1
        if i>= 50:
            test_sentences.append(row[0].lower())
            test_labels.append(0)
        else:
            train_sentences.append(row[0].lower())
            train_labels.append(0)
            



#### Review dataset generated

In [60]:
print("---------------------------------------------------------")
print(f"Training dataset {len(train_sentences)}")
print("---------------------------------------------------------")
# print(train_sentences)

print("---------------------------------------------------------")
print(f"Testing dataset: {len(test_sentences)}")
print("---------------------------------------------------------")
print(test_sentences)

---------------------------------------------------------
Training dataset 118
---------------------------------------------------------
---------------------------------------------------------
Testing dataset: 86
---------------------------------------------------------
['trajet de pleuville à memont', 'je veux aller de beaulieu à roquettes', 'aller-retour de bastelica à reviers', 'je veux aller de gare de baisieux à lugny', 'trajet de gare de colmar à gare de montferrand-thoraise', 'aller-retour de gare de onzain-chaumont-/-l à farébersviller (cité)', 'trajet de paris à aixe sur vienne', 'je veux aller de fignevelle à dessenheim', 'aller-retour de chaouilley à meolans revel', 'je veux aller de seilhan-le-bazert à gare de vic-mireval', 'trajet de gare de bréval à gare de crouy', 'aller-retour de marimont-lès-bénestroff à gare de seurre', 'trajet de fontaine la louvet à estaing', 'je veux aller de larressingle à cestas', 'aller-retour de argut dessus à lugy', 'je veux aller de gare de

In [71]:
print(shuffled_test_sentences[0])
print(shuffled_test_labels[0])
print(test_sentences[0])
print(test_labels[0])

la région de nord-pas-de-calais est célèbre pour ses ports et ses industries.
0
trajet de pleuville à memont
1


In [67]:
shuffled_test_sentences = test_sentences.copy()
random.shuffle(shuffled_test_sentences)

shuffled_test_labels = []
for sentence in shuffled_test_sentences:
    shuffled_test_labels.append(test_labels[test_sentences.index(sentence)])


### Training the model
Launch the training with instructions for SVM to look out for features based on the key words that was preprocessed earlier.
As the generated dataset revolve around sentences with similar structures and as we include the possible ways of asking for directions in actual real-life behaviour, we have determined to train our SVM with 4 specific set of keywords. 
("de depart", "aller-retour", "trajet de", "à destination")

In [36]:
# create feature vector for each sentence
features = np.array([[len(sentence), sentence.count("de depart"), sentence.count("aller-retour"), sentence.count("Trajet de"), sentence.count("à destination")] for sentence in train_sentences])

# create labels for each sentence
labels = np.array(train_labels) # 0 = not itinerary, 1 = itinerary

# create and train the SVM model
model = SVC(kernel="linear")
model.fit(features, labels)

SVC(kernel='linear')

### Testing the model

#### Pre-process datasets
We will label those destinations at best before feeding it to our model for training and prediction by detecting if therer are keywords like "aller", "retour", "de" and "à" which hints that the next words could concern a destination or a transiting point

In [130]:
# nlp = spacy.load("fr_core_news_lg")
# test = nlp("trajet de gare_de_colmar_st_joseph à gare_de_chatelaillon_plage")
# for token in test:
#     print(f"{token.text}:{token.pos_}")

In [37]:
list_of_frequent_words = ["aller", "retour", "à", "trajet"]
pos_to_replace = ["NOUN", "PROPN"]
def labelling_keywords(sentences):
    filtered_sentences = []
    nlp = spacy.load("fr_core_news_lg")
    corpus = nlp.pipe(sentences)
    for idx, doc in enumerate(corpus):
        sentence = ""
        doc_ents = OrderedDict()
        # for ent in doc.ents:
        #     doc_key = ent.text
        #     # print(f"{doc_key}:{ent.label_}")
        
        for token in doc:
            if token.text in nlp.vocab:
                # print(f"{idx} - {token.text}: {token.pos_}")
                if (token.text.lower() not in list_of_frequent_words) :
                    if token.text.lower() == "de" and doc[token.i-1].text.lower().strip() != "gare" and not "depart" in sentence:
                        sentence += f" {token.text.lower().strip()} "
                        if doc[token.i+1].pos_ in pos_to_replace:
                            sentence += "depart"
                        else:
                            sentence += f' {token.text.lower().strip()}'
                    elif doc[token.i-1].text.lower().strip() == "à" and token.pos_ in pos_to_replace:
                        sentence += " destination"
                    elif (doc[token.i-1].text.lower().strip() == "de" and doc[token.i-2].text.lower().strip() != "gare") :
                        sentence += ""
                    elif (doc[token.i-1].text.lower().strip() == "-"):
                        sentence += ""
                    elif token.text.lower().strip() == "-":
                        sentence += ""
                    elif (token.text.lower() == "de" and doc[token.i-1].text.lower().strip() == "gare"):
                        sentence += ""
                    else:
                        sentence += f' {token.text.lower().strip()}'
                else:
                    sentence += f' {token.text.lower().strip()}'
        # print(words_filtered)

        
        filtered_sentences.append(sentence)
    return filtered_sentences
        

Then we will test it with the test dataset underwent the same preprocessing. We will have SVM to look out for the same features as well. 

In [70]:
test_sentences_filtered = labelling_keywords(shuffled_test_sentences)
test_sentences_filtered

[' la région de depart est célèbre pour ses ports et ses industries .',
 ' aller retour de depart onzain à farébersviller ( cité )',
 ' mon amie vient de  de italie mais elle habite à destination depuis un an .',
 ' la ville de depart est un important centre commercial et économique en france .',
 ' je suis en train de  de la ville de depart .',
 ' je dois rendre mon travail à mon professeur demain .',
 ' le livre que je lis est de  de auteur célèbre j.k . rowling .',
 ' aller retour de depart le chateau à savigneux',
 ' la région de depart est célèbre pour ses forêts et ses montagnes .',
 " j' ai reçu une lettre de  de grand-père à destination .",
 ' trajet de depart colmar à destination montferrand',
 ' la ville de depart est connue pour ses monuments célèbres .',
 ' aller retour de depart valence à bricon(la poste )',
 ' mon prochain voyage sera à la gare marseille .',
 ' aller retour de depart à destination lesparre',
 ' je veux aller de depart à destination',
 ' la gare rennes est

In [72]:
# test the model on a new sentence
results = []
results_label = []

for test in test_sentences_filtered:
  test_features = np.array([[len(test), test.count("de depart"),  test.count("aller-retour"), test.count("trajet de"), test.count("à destination")]])
  prediction = model.predict(test_features)
  if prediction[0] == 0:
    results.append("The sentence is not an itinerary.")
  else:
    results.append("The sentence is an itinerary.")
  results_label.append(prediction[0])


#### Results of testing
Here we will present the descrepancies in terms of results between the correct labels and the guessed labels

In [73]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(shuffled_test_labels, results_label)
print(f"model's accuracy : {accuracy}")

model's accuracy : 0.9186046511627907


In [74]:
wrong_answers = 0
sentences_guessed_wrongly = []
wrong_labels = []
for index, result in enumerate(results_label):
    if result != shuffled_test_labels[index]:
        # print(f"result {result} | shuffled {shuffled_test_labels[index]}")
        wrong_answers=wrong_answers+1
        sentences_guessed_wrongly.append(shuffled_test_sentences[index])
        wrong_labels.append(result)
        # print(f"answer : {test_labels[index]}, predicted as {result}")
print(f"wrong answers: {wrong_answers}")
print(f"total sentences: {len(shuffled_test_labels)}")
print("sentences guessed wrongly:")
print("-----------------------------")
for index, sentence in enumerate(sentences_guessed_wrongly):
    print(f"{sentence} | was guessed as {wrong_labels[index]} and test label is {shuffled_test_labels[shuffled_test_sentences.index(sentence)]}")


wrong answers: 7
total sentences: 86
sentences guessed wrongly:
-----------------------------
aller-retour de gare de onzain-chaumont-/-l à farébersviller (cité) | was guessed as 0 and test label is 1
aller-retour de fontenoy le chateau à savigneux | was guessed as 0 and test label is 1
aller-retour de gare de valence-ville à bricon(la poste) | was guessed as 0 and test label is 1
j'ai resté à paris. | was guessed as 1 and test label is 0
je veux aller de coulonvillers à faverelles | was guessed as 0 and test label is 1
amènes-moi à toulouse. | was guessed as 1 and test label is 0
mon ami vient de france mais il habite à londres maintenant. | was guessed as 1 and test label is 0


### Testing with single spontaneous sentence
We provide an impromptu sentence for the prediction

In [75]:
single_test_sentence = ["à destination."]
single_test_sentence_filtered = labelling_keywords(single_test_sentence)
single_test_labels = [0]
results_label = []
for test in single_test_sentence_filtered:
  test_features = np.array([[len(test), test.count("de depart"), test.count("aller-retour"), test.count("Trajet de"), test.count("à destination")]])
  prediction = model.predict(test_features)
  if prediction[0] == 0:
    results.append("The sentence is not an itinerary.")
  else:
    results.append("The sentence is an itinerary.")
  results_label.append(prediction[0])

print(f"sentence tested: {single_test_sentence}")
print(f'The sentence should {"not" if single_test_labels[0] == 0 else ""} be an itinerary')
for result in results_label:
  print(f'guessed as: {"{is an itinerary}" if result == 1 else "{not an itinerary}"}')




sentence tested: ['à destination.']
The sentence should not be an itinerary
guessed as: {is an itinerary}


## Experiment with voice recognition component

In [77]:
data = []
with open('record_16-10-05.txt', encoding='UTF-8') as file_record:
  data.append(file_record.readline().strip())
print(data)
single_test_sentences=data
single_test_sentence_filtered = labelling_keywords(single_test_sentence)
results_label = []
for test in single_test_sentence_filtered:
  test_features = np.array([[len(test), test.count("de depart"), test.count("aller-retour"), test.count("Trajet de"), test.count("à destination")]])
  prediction = model.predict(test_features)
  if prediction[0] == 0:
    results.append("The sentence is not an itinerary.")
  else:
    results.append("The sentence is an itinerary.")
  results_label.append(prediction[0])

for result in results_label:
  print(f'guessed as: {"{is an itinerary}" if result == 1 else "{not an itinerary}"}')


['Je veux aller de Albi à Marseille.']
guessed as: {is an itinerary}


## Save trained model

In [14]:
import pickle

# Save the model to a file
with open('svm_model.pkl', 'wb') as f:
  pickle.dump(model, f)

## ---- testing with spacy -------

In [36]:
# departure = ""
# print("empty string") if not departure else print("not empty")
import spacy
nlp = spacy.load("fr_core_news_lg")
spacy.explain("SPACE")
test = nlp("Gaillac-12-N88/D95")
for token in test:
    print(f"text: {token.text} | pos: {token.pos_}")

text: Gaillac-12-N88 | pos: PROPN
text: / | pos: ADP
text: D95 | pos: SPACE
